In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import sys
sys.modules['Image'] = Image 
import pandas as pd
import numpy as np
import cv2
import os

In [ ]:
# Globals   
path_to_data_dir = "../../database/sorted_by_state/"
test_file = "Arkansas/_09TL3yHwqMzcB8m4BUGoQ.jpeg"

In [49]:
data = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [50]:
train_dataset = data.flow_from_directory(path_to_data_dir, target_size=(224, 224), batch_size=64, class_mode='categorical', shuffle=True, subset="training")
validation_dataset = data.flow_from_directory(path_to_data_dir, target_size=(224, 224), batch_size=64, class_mode='categorical', shuffle=False, subset="validation")

Found 834 images belonging to 1 classes.
Found 208 images belonging to 1 classes.


In [53]:
# model with 3 convolution layers max pooled 3 times with relu as activation function and softmax applied to output layer
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=None, padding='valid'),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu', use_bias=True),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=None, padding='valid'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu', use_bias=True),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=None, padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])


model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=RMSprop(learning_rate=0.001),
              metrics=[
              tf.keras.metrics.Accuracy(),
              tf.keras.metrics.Recall(),
              tf.keras.metrics.Precision(),
              tf.keras.metrics.F1Score(),
            ],
              )

In [ ]:
from tensorflow.keras.callbacks import Callback

epochs_to_save = [1,2]
model_version= "v1.0"

save_dir = "saved_models/"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

class CustomClallback(Callback):
    def __init__(self, epochs_to_save):
        super().__init__()
        self.epochs_to_save = epochs_to_save

    def on_epoch_end(self, epoch, logs=None):
        if epoch in self.epochs_to_save:
            fn = f'{model_version}_{epoch:02d}.keras'

            # Save the model weights
            self.model.save(save_dir + fn)
            print(f"Model weights saved at epoch {epoch} to {fn}")


custom_callback = CustomClallback(epochs_to_save=epochs_to_save)

model_fit = model.fit(train_dataset, 
                    epochs=epochs_to_save[-1] + 1, #+1 to include the last epoch
                    validation_data=validation_dataset,
                    callbacks=[custom_callback],
                    )

for epoch in epochs_to_save:
    fn = f'{model_version}_{epoch:02d}.keras'
    model.load_weights(save_dir + fn)

    # Evaluate the model on the validation dataset
    accuracy, recall, precision, f1_score = model.evaluate(validation_dataset)
    print(f"Model {fn} - Validation Accuracy: {accuracy}, Validation Precision: {precision}, Validation Recall: {recall}, Validation f1_score: {f1_score}")

Epoch 1/3


f:\School\MSU Sm 5\CSE-404\CSE404-Group-Project\.venv\Lib\site-packages\keras\src\ops\nn.py:908: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
f:\School\MSU Sm 5\CSE-404\CSE404-Group-Project\.venv\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 683ms/step - accuracy: 1.0000 - f1_score: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000

f:\School\MSU Sm 5\CSE-404\CSE404-Group-Project\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 15s 852ms/step - accuracy: 1.0000 - f1_score: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 1.0000 - val_f1_score: 1.0000 - val_loss: 0.0000e+00 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/3
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 754ms/step - accuracy: 1.0000 - f1_score: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000Model weights saved at epoch 1 to v1.0_01.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 1.0000 - f1_score: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 1.0000 - val_f1_score: 1.0000 - val_loss: 0.0000e+00 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/3
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 757ms/step - accuracy: 1.0000 - f1_score: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000Model weights saved at epoch 2 to v1.0_02.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 1.0000 - f1_score: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - r

ValueError: too many values to unpack (expected 2)